# TensorFlow入门

In [1]:
import tensorflow as tf

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 计算模式--计算图

In [2]:
a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([1.0, 2.0], name='b')
result = a + b
print(result)

Tensor("add:0", shape=(2,), dtype=float32)


In [3]:
# 如果没有特殊声明，a.graph返回其所属的计算图，即默认的计算图
print(a.graph == tf.get_default_graph())

True


## 数据模式--张量

In [4]:
a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([2.0, 3.0], name='b')
result = tf.add(a, b, name='add')
#print(result)

# 获取结果的维度
print(result)

Tensor("add_1:0", shape=(2,), dtype=float32)


In [5]:
tf.Session().run(result)

array([3., 5.], dtype=float32)

## 运行模式--会话

## Session

In [6]:
# 创建一个会话
sess = tf.Session()
print(sess.run(result))
# 关闭会话
sess.close()

[3. 5.]


In [7]:
# 通过Python的上下文管理器来管理会话
with tf.Session() as sess:
    print(sess.run(result))

[3. 5.]


In [8]:
# 通过Python的上下文管理器来管理会话
sess = tf.Session()
with sess.as_default():
    print(result.eval())

[3. 5.]


In [9]:
sess = tf.Session()
print(sess.run(result))
print(result.eval(session=sess))

[3. 5.]
[3. 5.]


In [10]:
# tf.InteractiveSession
sess = tf.InteractiveSession()
print(result.eval())

[3. 5.]


## Placeholder

In [11]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)

In [12]:
print(sess.run(adder_node, {a: 3, b:4.5}))
print(sess.run(adder_node, {a: [1,3], b: [2, 4]}))

7.5
[3. 7.]


In [13]:
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b:4.5}))

22.5


## Variable

In [14]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b

In [15]:
init = tf.global_variables_initializer()
sess.run(init)

In [16]:
print(sess.run(linear_model, {x:[1,2,3,4]}))

[0.         0.3        0.6        0.90000004]


In [17]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

23.66


In [18]:
fixW = tf.assign(W, [-1])
fixb = tf.assign(b, [1])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


## tf.train

In [19]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [20]:
sess.run(init) # 重新设置初始化参数
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([0.9999908], dtype=float32)]


## TensorFlow实现线性回归

In [21]:
import numpy as np
import tensorflow as tf

# 模型参数
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# 模型的输入和输出
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# 定义损失函数
loss = tf.reduce_sum(tf.square(linear_model - y)) # 均方误差

# 优化算法
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# 训练数据
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]

# 训练过程
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# 训练误差
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [0.9999908] loss: 5.6999738e-11
